# Morphosyntactic tagging

In [1]:
import requests
import os
import math

In [2]:
files_names = []
for file_name in os.listdir("../../ustawy"):
    if file_name.endswith(".txt"):
        files_names.append(os.path.join("../../ustawy", file_name))

In [3]:
bigrams = {}
tags = {}

In [4]:
for file_name in files_names:
    with open(file_name, encoding='utf-8') as file:
        bill = file.read()
        bill = bill.lower()
        
        results = requests.post('http://localhost:9200', data=bill.encode('UTF-8')).text
        
        lines = results.splitlines()
        
        prev_tag = None
        for line in lines:
            words = line.split()
            if len(words) == 3:
                word = words[0]
                tag = words[1].split(':')[0]
                tags[(word, tag)] = tags.get((word, tag),0) + 1
                
                if prev_tag is not None:
                    bigrams[(prev_tag,(word, tag))] = bigrams.get((prev_tag,(word, tag)),0) + 1
                    
                prev_tag = (word, tag)


In [6]:
len(bigrams)

483827

In [8]:
key_to_delete = []
for key in bigrams.keys():
    if not key[0][0].isalpha() or not key[1][0].isalpha():
        key_to_delete.append(key)
        
for key in key_to_delete:
    bigrams.pop(key, None)

In [9]:
len(bigrams)

365008

## LLR

In [10]:
def H(args, N):
    res = 0
    for k in args:
        k /= N
        k_eq_0 = 1 if k == 0 else 0
        res += k * math.log(k + k_eq_0)
    return res

In [11]:
bigrams_LLR = {}
bigrams_occurs = sum(bigrams.values())

In [12]:
for bigram, AandB in bigrams.items():
    (word1, tag1), (word2, tag2) = bigram
    BnoA = tags[(word2, tag2)] - AandB
    AnoB = tags[(word1, tag1)] - AandB
    noAnoB = bigrams_occurs - (AandB + BnoA + AnoB)
    LLR = 2 * bigrams_occurs * (H([AandB, BnoA, AnoB, noAnoB], bigrams_occurs) - \
                               H([AandB + BnoA, AnoB + noAnoB], bigrams_occurs) - \
                               H([AandB + AnoB, BnoA + noAnoB], bigrams_occurs))
    bigrams_LLR[bigram] = LLR     

In [13]:
bigrams_LLR = {k: v for k, v in sorted(bigrams_LLR.items(), key=lambda item: item[1], reverse=True)}

In [14]:
for bigram, llr in list(bigrams_LLR.items())[:10]:
    print(str(bigram) + ' ' + str(llr))

(('który', 'adj'), ('mowa', 'subst')) 248323.47813489943
(('o', 'prep'), ('który', 'adj')) 163662.8380830866
(('mowa', 'subst'), ('w', 'prep')) 149478.78490378824
(('otrzymywać', 'fin'), ('brzmienie', 'subst')) 111105.82963924854
(('minister', 'subst'), ('właściwy', 'adj')) 67763.46033475136
(('w', 'prep'), ('artykuł', 'brev')) 67751.07338376575
(('dodawać', 'fin'), ('się', 'qub')) 66927.24815087883
(('w', 'prep'), ('ustęp', 'brev')) 56401.460472486884
(('stosować', 'fin'), ('się', 'qub')) 52971.0157973974
(('droga', 'subst'), ('rozporządzenie', 'subst')) 51680.8027317483


In [15]:
bigrams_part = {}
for bigram, freq in bigrams.items():
    (word1, tag1), (word2, tag2) = bigram
    bigrams_part[(tag1, tag2)] = bigrams_part.get((tag1, tag2), 0) + freq

In [16]:
bigrams_part = {k: v for k, v in sorted(bigrams_part.items(), key=lambda item: item[1], reverse=True)}

In [17]:
for bigram, llr in list(bigrams_part.items())[:10]:
    print(str(bigram) + ' ' + str(llr))

('prep', 'subst') 327378
('subst', 'subst') 290104
('subst', 'adj') 274761
('adj', 'subst') 188238
('subst', 'prep') 173254
('subst', 'conj') 85145
('conj', 'subst') 84393
('prep', 'adj') 79459
('ger', 'subst') 77510
('prep', 'brev') 67230


In [18]:
bigrams_part_top5 = {}

In [19]:
for category in list(bigrams_part.keys())[:10]:
    cat_tag1, cat_tag2 = category
    bigrams_part_top5[category] = [(word1, word2) for ((word1,tag1), (word2,tag2)) in list(bigrams.keys()) 
                                   if tag1 == cat_tag1 and tag2 == cat_tag2][:5]

In [20]:
list(bigrams_part_top5.items())

[(('prep', 'subst'),
  [('z', 'dzień'),
   ('o', 'zmiana'),
   ('w', 'ustawa'),
   ('przez', 'osoba'),
   ('poza', 'zakres')]),
 (('subst', 'subst'),
  [('zmiana', 'ustawa'),
   ('dzień', 'ogłoszenie'),
   ('wzór', 'odznaka'),
   ('pomoc', 'osoba'),
   ('minister', 'sprawa')]),
 (('subst', 'adj'),
  [('straż', 'pożarny'),
   ('szkoła', 'państwowy'),
   ('osoba', 'prawny'),
   ('zakres', 'ustawowy'),
   ('środek', 'finansowy')]),
 (('adj', 'subst'),
  [('państwowy', 'straż'),
   ('ustawowy', 'zadanie'),
   ('właściwy', 'zleceniodawca'),
   ('ten', 'tytuł'),
   ('specjalny', 'jednostka')]),
 (('subst', 'prep'),
  [('ustawa', 'z'),
   ('ustawa', 'o'),
   ('zleceniodawca', 'na'),
   ('życie', 'po'),
   ('dzień', 'od')]),
 (('subst', 'conj'),
  [('zasada', 'i'),
   ('życie', 'albo'),
   ('zdrowie', 'lub'),
   ('funkcjonariusz', 'oraz'),
   ('warunek', 'i')]),
 (('conj', 'subst'),
  [('a', 'środek'),
   ('i', 'tryb'),
   ('lub', 'szkoda'),
   ('i', 'sposób'),
   ('i', 'przetwór')]),
 (('prep

# Opowiedzi

#### 1) What types of bigrams have been found?

W zakresie najczęściej powtarzających się par kategorii znalazły się zaimki z rzeczownikami i na odwrót (np. w dniu, ustawa o), podwójne rzeczowniki (np. zmiana ustawy), rzeczowniki z przymiotnikami i na odwrót (np. straż pożarna, państwowa straż), rzeczowniki ze spójnikami i na odwrót (np. zasada i, a środek), zaimki z przymiotnikami (np. o państwowy), rzeczownik odczasownikowy z rzeczownikiem (np. wykonywać zadanie) oraz zaimki ze skrótowcami (np. w art). Zdecydowanie rzeczowniki przeważają w najczęściej pojawiających się kolokacjach, a po nich najczęstsze są także zaimki.

#### 2) Which of the category-pairs indicate valuable multiword expressions? Do they have anything in common?

Najwięcej wnoszą kategorie rzeczownik+rzeczownik, rzeczownik+przymiotnik oraz przymiotnik+rzeczownik. Ta pierwsza grupa zawiera informacje takie jak 'minister spraw', czyli zawiera m.in. nazwy własne, z czego można wywnioskować kogo lub czego dotyczy ustawa. W drugiej grupie zawarły się kolokacje takie jak 'straż pożarna', czyli otrzymujemy znów informacje kogo/czego może dotyczyć ustawa. W ostatniej grupie znalazły się podobne kolokacje jak w poprzedniej kategorii, ale są to takie nazwy, w których stawiamy przymiotnik przed rzeczownikiem, czyli np. 'państwowa straż'. Ta ostatnia kategoria jest najmniej sensowna z wymienionych, bo będzie zawierać wszystkie opisy rzeczowników, które mogą nic nie wnosić. Cechą wspólną tych grup jest to, że wszystkie zawierają rzeczownik i potrafią opisywać nazwy własne.

#### 3) Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?

W tym wypadku połączenie tych metod (zkategoryzowanie i na podstawie LLR wyciągnięcie 5 pierwszych bigramów) okazało się dostarczyć najwięcej informacji o tym, czego i kogo dotyczy ustawa. Dzięki kategoriom możemy ograniczyć analizę tylko do tych typów, które rzeczywiście coś wnoszą i pominąć np. spójniki, których zwykle jest bardzo dużo przez co analiza trwa dłużej i nic nie dodają w kwestii tematu. LLR pozwala natomiast wyłuskać najczęściej pojawiające się kolokacje, co pozwala określić główny temat tekstu. 

#### 4) Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world problem?

Kategoryzacja tekstu pod względem tematyki (np. polityka, nauka), można określić, jak bardzo tekst dotyczy danej dziedziny, co można użyć np. w wyszukiwarkach internetowych. Innym zastosowaniem może być automatyczne tagowanie tekstu albo podpowiadanie kolejnego słowa w edytorach tekstu.